전체적인 과정
============

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
from xgboost import XGBRegressor

In [2]:
#import files data
Raw = pd.read_csv("Auction_Master_en/Auction_master_train_en.csv") #1933 x 32 columns
test_raw = pd.read_csv("Auction_Master_en/Auction_master_test_en.csv")
submit=pd.read_csv("Auction_Master_en/Auction_submission_en.csv")

In [3]:
#NaN data processing -> sepcific & close_result has null data
Raw.isnull().sum()

Auction_key                       0
Auction_class                     0
Bid_class                         0
Claim_price                       0
Appraisal_date                    0
Auction_count                     0
Appraisal_company                 0
Auction_miscarriage_count         0
Total_land_gross_area             0
Total_land_real_area              0
Total_land_auction_area           0
Total_building_area               0
Total_building_auction_area       0
Total_appraisal_price             0
Minimum_sales_price               0
First_auction_date                0
Final_auction_date                0
Final_result                      0
Creditor                          0
addr_etc                          0
Apartment_usage                   0
Preserve_regist_date              0
Total_floor                       0
Current_floor                     0
Share_auction_YorN                0
Specific                       1869
Close_date                        0
Close_result                

In [4]:
Raw['Close_result']=Raw['Close_result'].fillna('not dividend') #NaN 값 채우기

In [5]:
#Checking Columns
Raw.columns

Index(['Auction_key', 'Auction_class', 'Bid_class', 'Claim_price',
       'Appraisal_date', 'Auction_count', 'Appraisal_company',
       'Auction_miscarriage_count', 'Total_land_gross_area',
       'Total_land_real_area', 'Total_land_auction_area',
       'Total_building_area', 'Total_building_auction_area',
       'Total_appraisal_price', 'Minimum_sales_price', 'First_auction_date',
       'Final_auction_date', 'Final_result', 'Creditor', 'addr_etc',
       'Apartment_usage', 'Preserve_regist_date', 'Total_floor',
       'Current_floor', 'Share_auction_YorN', 'Specific', 'Close_date',
       'Close_result', 'point.y', 'point.x', 'addr_en', 'Hammer_price'],
      dtype='object')

원핫레이블 사용할 데이터들
============
1.Auction_class 2가지 값 가짐.(random,forced)  
2.Bid_class 3가지 값 가짐. (individual, general, bundle)  
3.Creditor 2가지 값 가짐.(company or group, private)  
4.Apartment_usage 2가지 값 가짐.(Apartment & stores, Apartment)  

In [6]:
One_hot_train = pd.DataFrame()
One_hot_train= Raw.loc[:,('Auction_class', 'Creditor', 'Apartment_usage')]
encoder.fit(One_hot_train['Auction_class'])
One_hot_train['Auction_class'] = encoder.transform(One_hot_train['Auction_class'])
encoder.fit(One_hot_train['Creditor'])
One_hot_train['Creditor'] = encoder.transform(One_hot_train['Creditor'])
encoder.fit(One_hot_train['Apartment_usage'])
One_hot_train['Apartment_usage'] = encoder.transform(One_hot_train['Apartment_usage'])
one_hot =pd.get_dummies(Raw['Bid_class'])
One_hot_train =pd.concat([One_hot_train, one_hot],axis=1)
One_hot_train

,Auction_class,Creditor,Apartment_usage,bundle,general,individual
0,1,0,1,0,0,1
1,1,0,0,0,1,0
2,1,0,0,0,0,1
3,1,0,0,0,1,0
4,0,1,0,0,1,0
...,...,...,...,...,...,...
1928,0,1,0,0,1,0
1929,1,0,0,0,1,0
1930,1,0,1,0,1,0
1931,0,1,0,0,1,0


조정해야할만한 데이터들
============
  1.Columns related with address(addr_etc, point.y, point.x, addr_en)   
  2.Final_result -> 전부 성공적인 경매... unique 사용시 결과: array(['successful bid'], dtype=object)  
  3.Specific -> 거의 비어있는 값  
  4.Total_land_real_area/Total_land_gross_area * Total_land_auction_area  
  5.Total_building_auction_area/Total_building_area  
  6.Appraisal_date-> 평가일자가 영향을 준다고 보기 어려움.  
  7.Currnet_floor/Total_floor  

In [7]:
Train_scaled = pd.DataFrame()
Train_scaled['longitude'] = Raw['point.x']
Train_scaled['latitude'] = Raw['point.y']
Train_scaled['Ratio_land']= Raw['Total_land_real_area']/Raw["Total_land_gross_area"]*Raw['Total_land_auction_area']
Train_scaled['Ratio_building'] = Raw['Total_building_auction_area']/Raw['Total_building_area']
Train_scaled['Ratio_floor']= Raw['Current_floor']/Raw['Total_floor']
Train_scaled

,longitude,latitude,Ratio_land,Ratio_building,Ratio_floor
0,129.137048,35.162717,0.110786,1.0,0.111111
1,129.145068,35.156633,0.008285,1.0,0.728571
2,128.996765,35.184601,0.033678,1.0,0.933333
3,129.089081,35.154180,0.044568,1.0,0.550000
4,128.998874,35.099630,2.637251,1.0,1.000000
...,...,...,...,...,...
1928,127.048683,37.648811,0.024756,1.0,0.714286
1929,127.077063,37.663291,0.076743,1.0,1.000000
1930,126.981994,37.558319,0.048539,1.0,0.593750
1931,127.028002,37.647061,0.105526,1.0,0.733333


train 데이터셋 완성하기
============

In [11]:
train_x = pd.DataFrame()
train_x = pd.concat([One_hot_train, Train_scaled],axis=1)
train_y = Raw['Hammer_price']
train_x

,Auction_class,Creditor,Apartment_usage,bundle,general,individual,longitude,latitude,Ratio_land,Ratio_building,Ratio_floor
0,1,0,1,0,0,1,129.137048,35.162717,0.110786,1.0,0.111111
1,1,0,0,0,1,0,129.145068,35.156633,0.008285,1.0,0.728571
2,1,0,0,0,0,1,128.996765,35.184601,0.033678,1.0,0.933333
3,1,0,0,0,1,0,129.089081,35.154180,0.044568,1.0,0.550000
4,0,1,0,0,1,0,128.998874,35.099630,2.637251,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
1928,0,1,0,0,1,0,127.048683,37.648811,0.024756,1.0,0.714286
1929,1,0,0,0,1,0,127.077063,37.663291,0.076743,1.0,1.000000
1930,1,0,1,0,1,0,126.981994,37.558319,0.048539,1.0,0.593750
1931,0,1,0,0,1,0,127.028002,37.647061,0.105526,1.0,0.733333


In [16]:
model = XGBRegressor(n_estimators = 500, learning_rate = 0.02)
model.fit(train_x, train_y, verbose = False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

test 데이터셋 가공하기
============

In [17]:
test_raw['Close_result']=test_raw['Close_result'].fillna('not dividend') #NaN 값 채우기

In [18]:
One_hot_test = pd.DataFrame()
One_hot_test= test_raw.loc[:,('Auction_class', 'Creditor', 'Apartment_usage')]
encoder.fit(One_hot_test['Auction_class'])
One_hot_test['Auction_class'] = encoder.transform(One_hot_test['Auction_class'])
encoder.fit(One_hot_test['Creditor'])
One_hot_test['Creditor'] = encoder.transform(One_hot_test['Creditor'])
encoder.fit(One_hot_test['Apartment_usage'])
One_hot_test['Apartment_usage'] = encoder.transform(One_hot_test['Apartment_usage'])
one_hot =pd.get_dummies(test_raw['Bid_class'])
One_hot_test =pd.concat([One_hot_test, one_hot],axis=1)
One_hot_test

,Auction_class,Creditor,Apartment_usage,bundle,general,individual
0,0,1,1,0,0,1
1,0,1,1,0,0,1
2,0,1,1,0,0,1
3,0,1,1,0,0,1
4,0,1,1,0,0,1
...,...,...,...,...,...,...
824,0,0,1,0,0,1
825,0,0,0,0,1,0
826,0,1,0,0,1,0
827,1,0,0,0,1,0


In [19]:
Test_scaled = pd.DataFrame()
Test_scaled['longitude'] = test_raw['point.x']
Test_scaled['latitude'] = test_raw['point.y']
Test_scaled['Ratio_land']= test_raw['Total_land_real_area']/test_raw["Total_land_gross_area"]*test_raw['Total_land_auction_area']
Test_scaled['Ratio_building'] = test_raw['Total_building_auction_area']/test_raw['Total_building_area']
Test_scaled['Ratio_floor']= test_raw['Current_floor']/test_raw['Total_floor']
Test_scaled

,longitude,latitude,Ratio_land,Ratio_building,Ratio_floor
0,129.088687,35.201112,0.624001,1.0,0.333333
1,129.088687,35.201112,0.521095,1.0,0.400000
2,129.088687,35.201112,0.624001,1.0,0.533333
3,129.088687,35.201112,0.624001,1.0,0.600000
4,129.088687,35.201112,0.624001,1.0,0.733333
...,...,...,...,...,...
824,127.027309,37.605407,0.055586,1.0,1.000000
825,127.039551,37.663305,0.390407,1.0,0.214286
826,127.091109,37.614529,0.020143,1.0,0.416667
827,127.047071,37.620359,0.065306,1.0,0.941176


In [20]:
test_x = pd.DataFrame()
test_x = pd.concat([One_hot_test, Test_scaled],axis=1)
test_x

,Auction_class,Creditor,Apartment_usage,bundle,general,individual,longitude,latitude,Ratio_land,Ratio_building,Ratio_floor
0,0,1,1,0,0,1,129.088687,35.201112,0.624001,1.0,0.333333
1,0,1,1,0,0,1,129.088687,35.201112,0.521095,1.0,0.400000
2,0,1,1,0,0,1,129.088687,35.201112,0.624001,1.0,0.533333
3,0,1,1,0,0,1,129.088687,35.201112,0.624001,1.0,0.600000
4,0,1,1,0,0,1,129.088687,35.201112,0.624001,1.0,0.733333
...,...,...,...,...,...,...,...,...,...,...,...
824,0,0,1,0,0,1,127.027309,37.605407,0.055586,1.0,1.000000
825,0,0,0,0,1,0,127.039551,37.663305,0.390407,1.0,0.214286
826,0,1,0,0,1,0,127.091109,37.614529,0.020143,1.0,0.416667
827,1,0,0,0,1,0,127.047071,37.620359,0.065306,1.0,0.941176


In [22]:
prediction = model.predict(test_x)

In [23]:
submit['Hammer_price']= prediction
submit.to_csv('submission.csv', index = False)